# Creating new Opacity Databases 

``PICASO`` currently comes with one set of molecular and continuum opacities. The continuum opacity file is already included in the `reference/` directory. The molecular opacity file is included on [Open Science Framework](https://osf.io/hf8bp/). 

If you want to swap out opacities, this tutorial shows you how to make a correctly formatted `json` file that will compatible with `PICASO`

Here you will learn how to create the correctly formatted database that ``picaso`` reads in for the **molecular.json** and **continuum.json**

In [1]:
import json
import numpy as np
import os

## Continuum Opacity Formatting : ``continuum.json``

**Note on molecule names**: Because ``picaso`` uses dict formatting to handle opacities, users can easily swap in different molecules. 

For example, if I wanted to include CO2-H2 CIA absorption, I can add ``CO2H2`` to the molecules list below. 

**Exceptions**: The exceptions to this are non-CIA continuum opacities. Right now, the other sources of continuum enabled are ``H2-``, ``H-bf`` and ``H-ff`` which have odd-ball formatting since they aren't simple two molecules. _Please let me know if you want to see another continuum source added_.

**Careful** with case sensitive molecules like **TiO**, **Na**. Make sure you get these right. 

In [2]:
wave_unit = 'cm-1'
opacity_unit = 'cm-1 amagat-2'
molecules = ['H2H2','H2He','H2CH4','H2-','H-bf','H-ff'] #and whatever else CO2H2
temperatures = [300, 500, 1000, 2000] #Kelvin

## Create Fake Continuum Dataset

In [3]:
#start by filling in molcules for dict 
db_continuum = {m:{} for m in molecules}

#add meta data
db_continuum['wave_units'] =wave_unit
db_continuum['opacity_unit'] =opacity_unit
db_continuum['temperature_unit'] ='K'
db_continuum['wavenumber_grid'] = list(np.linspace(20,30000,1000))#fake grid, make sure it's a list 

#add data
fake_absorption = np.zeros(1000)+1e-30 #cm-1 amagat-2
for m in molecules:
    for t in temperatures: 
        db_continuum[m][str(t)] = list(fake_absorption)

## Quick json Tutorial

Let's make sure you know how to navigate your file

In [4]:
#what molecules are in your database? 
print([i for i in db_continuum.keys()])

['H2H2', 'H2He', 'H2CH4', 'H2-', 'H-bf', 'H-ff', 'wave_units', 'opacity_unit', 'temperature_unit', 'wavenumber_grid']


In [5]:
#what temperatures are in your database? 
print([i for i in db_continuum['H2CH4'].keys()])

['300', '500', '1000', '2000']


In [6]:
#check to see if there is your temperature of interest
opa = db_continuum.get('H2H2').get('300')
print(len(opa))

1000


In [7]:
#get wavelength grid 
wave = db_continuum['wavenumber_grid']

## Molecular Opacity Formatting: ``molecular.json``

Because ``picaso`` uses ``dict`` formatting to handle opacities, users can add **any** new molecule to this database.

**Careful** with case sensitive molecules like **TiO**, **Na**. Make sure you get these right. 

**Wave Grid** must be the same as **continuum.json**

In [8]:
wave_unit = 'cm-1'
opacity_unit = 'cm-2/g'
molecules = ['H2','H','He','CH4','H2O','N2','TiO'] #and whatever else
temperatures = [300, 500, 1000, 2000] #Kelvin
pressures = [1e-6, 1, 10] #bars
prssure_unit = 'bars'
temperature_unit = 'K'

### Create Molecular Dataset

In [9]:
#initialize db with molcules and temps 
db_molecular = {m:{str(t):{} for t in temperatures} for m in molecules}

#add meta data
db_molecular['wave_units'] =wave_unit
db_molecular['opacity_unit'] =opacity_unit
db_molecular['prssure_unit'] =prssure_unit
db_molecular['temperature_unit'] =temperature_unit
db_molecular['wavenumber_grid'] = db_continuum['wavenumber_grid']#MUST be on same grid as continuum db!!!

#add data
fake_opacity = np.zeros(1000)+1e-30 #cm-2/g
for m in molecules:
    for t in temperatures: 
        for p in pressures:
            db_molecular[m][str(t)][str(p)] = list(fake_opacity) 